In [1]:
import numpy as np
import pandas as pd
import pathlib
import os.path

In [2]:
df_results = pd.read_json(
    os.path.join(str(pathlib.Path.home()), "fs/daaf/exp/reward-recovery/1708735081-report.json")
)

In [3]:
df_results

,spec,least,ols-em
0,"{'name': 'FrozenLake-v1', 'args': {'is_slipper...","[[1e-10, -1.4000000000000001e-09, -7.000000000...","[[None, None, None, None], [None, None, None, ..."
1,"{'name': 'RedGreenSeq', 'args': {'cure': ['red...","[[-0.9999997616, -2.0000007153, -1.99999928469...","[[-0.9999998764, -2.0000000622, -2.0000000216]..."
2,"{'name': 'ABCSeq', 'args': {'length': 7}}","[[-0.9999991655, -2.0000002384, -3.0000026226,...","[[-1.0000000095, -1.9999999988, -2.9999999976,..."
3,"{'name': 'IceWorld', 'args': {'map_name': '4x4'}}","[[-1.0000002384, -0.9999994040000001, -0.99999...","[[-1.0000007891, -1.0000016741, -1.0000006982,..."
4,"{'name': 'GridWorld', 'args': {'grid': 'oooooo...","[[-1.0000007153, -0.9999993443, -0.9999995828,...","[[-1.0, -0.9999999982000001, -1.0, -0.99999999..."
5,"{'name': 'TowerOfHanoi', 'args': {'num_disks':...","[[-1.0000001192, -0.9999997616, -2.0000009537,...","[[-1.0, -1.0, -2.0, -2.0, -2.0, -2.0], [-0.999..."


In [4]:
for row in df_results.to_dict('records'):
    name = row["spec"]["name"]
    args = row["spec"]["args"]
    least = np.array(row["least"], dtype=np.float64)
    ols_em = np.array(row["ols-em"], dtype=np.float64)
    if np.sum(np.isnan(least)) > 0 or np.sum(np.isnan(ols_em)) > 0:
        continue
    mae = np.sum(np.abs(least - ols_em)) / len(least)
    equality = np.allclose(least, ols_em, atol=1e-02)
    print(f"env: {name}({args}): {equality}; MeanAbsError: {mae}")
    # 

env: RedGreenSeq({'cure': ['red', 'green', 'wait', 'green', 'red', 'red', 'green', 'wait']}): True; MeanAbsError: 0.00011085625555561268
env: ABCSeq({'length': 7}): True; MeanAbsError: 0.00047056320000015783
env: IceWorld({'map_name': '4x4'}): True; MeanAbsError: 0.0011881916562505072
env: GridWorld({'grid': 'oooooooooooo\noooooooooooo\noooooooooooo\nsxxxxxxxxxxg'}): True; MeanAbsError: 4.7882594737652565e-05
env: TowerOfHanoi({'num_disks': 4}): True; MeanAbsError: 3.478436913579815e-05
